In [1]:
from hello import hello, hello3
from officelib.xllib import *
from officelib import const

In [3]:
HERR = (hello.HelloError, hello3.HelloError)

from time import sleep
def set_mg(sp):
    try:
        app.setmg(1, sp)
    except HERR:
        app.login()
        app.setmg(1, sp)
        
def update_ss(cell, val):
    cell.Value = val
    c = cell.Offset(2, 1)
    c.Select()
    return c
        
def frange(start, stop, step):
    i = 0
    v = start + i * step
    while v < stop:
        yield v
        i += 1
        v = start + i * step
        

def run_test(name, mfc_max, c=None):
    tmin = mfc_max / 10
    print("Testing ", name, "MFC: %.1f LPM Max" % mfc_max)
    app.login()
    for fr in frange(tmin, mfc_max+tmin/2, tmin):
        set_mg(fr)
        v = input("%.2f LPM:" % fr)
        if v.lower() in ("break", "quit", "q", "stop"):
            break
        c = update_ss(c, v)

def air_on():
    app.setph(2,0,0)
    app.setdo(2,0,0)
    
def n2_on():
    app.setph(2,0,0)
    app.setdo(1, 100, 0)
    
def o2_on():
    app.setph(2,0,0)
    app.setdo(1, 0, 100)
    
def co2_on():
    app.setconfig("pH", "CO2_Manual_Max_(%)", 100)
    app.setph(1, 100, 0)
    app.setdo(2, 0, 0)
    app.setmg(1, 0.1)
    input("Purging line of air. Press enter when done.")
    

cc = {
    "co2": 4,
    "air": 1,
    "n2": 2,
    "o2": 3
}

on_fn = {
    "co2": co2_on,
    "air": air_on,
    "n2": n2_on,
    "o2": o2_on
}

    
def s():
    return xl.Selection

def run(name, mfcm):
    c = cc[name]
    app.login()
    if name == "co2":
        co2m = app.getConfig()['pH']['CO2_Manual_Max_(%)']
    on_fn[name]()
    run_test(name, mfcm, cells(2,c))
    if name == "co2":
        app.login()
        app.setconfig("pH", "CO2_Manual_Max_(%)", co2m)

def all_off():
    app.login()
    app.setag(0, 12)
    app.setpumpa(0, 0)
    app.setpumpb(0, 0)
    app.setpumpc(0, 0)
    app.setpumpsample(0, 0)
    
def connect():
    global app
    v = input("Version: ").lower()
    ip = input("Enter IP Address: ")
    if v == "3":
        app = hello3.HelloApp(ip)
    else:
        app = hello.HelloApp(ip)
    app.getConfig = types.MethodType(getConfig, app)
    
def init():
    global xl, wb, ws, cells, cell_range
    xl, wb, ws, cells = xlObjs()
    cell_range = cells.Range
    cell_range("A1:D1").Value = [("Air", "N2", "O2", "CO2")]
    

In [7]:
from hello.hello3 import HelloXML
def getConfig(self):
    query = "?&call=getConfig"
    rsp = self.send_request(query)
    xml = HelloXML(rsp)
    self._verify_xml(xml)
    # James renamed the clusters to SV_Air
    # and SV_Mag to make it easier for him
    # to keep track of. The "proper" solution
    # would be to just return the proper XML 
    # element when parsing. The easiest solution
    # is to just check for the key, since conversion
    # from XML to dict occurs upstream. 
    try:
        return xml.data['System_Variables_Mag']
    except KeyError:
        return xml.data['System_Variables']
import types

In [5]:
init()
connect()
all_off()

Version: 3
Enter IP Address: 192.168.1.13


In [ ]:
run("air", .5)

In [ ]:
run("n2", 0.5)

In [ ]:
run("o2", .5)

In [8]:
run("co2", .3)

Purging line of air. Press enter when done.
Testing  co2 MFC: 0.3 LPM Max
0.03 LPM:30.5
0.06 LPM:61.5
0.09 LPM:92.3
0.12 LPM:b
0.15 LPM:q
